<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
np.random.seed(42)

## Градиентный спуск: повторение

Рассмотрим функцию от двух переменных:

In [ ]:
def f(x):
    """
    param x: np.array(np.float) - массив размерности 2
    return: np.float
    """

    return np.sum(np.sin(x)**2, axis=0)

Обратите внимание, что $x$ - `numpy.array` размерности $2$.

**Напоминание.**

Что мы хотим? Мы хотим найти минимум этой функции (в машинном обучении мы обычно хотим найти минимум **функции потерь**, например, **MSE**), а точнее найти $x_1$ и $x_2$ такие, что при них значение $f(x_1,x_2)$ минимально, то есть **точку экстремума**. 

Как мы будем искать эту точку? Используем методы оптимизации (в нашем случае - минимизации). Одним из таких методов и является **градиентный спуск**.

Реализуем функцию, которая будет осуществлять градиентный спуск для функции $f$.

**Примечание:** вам нужно посчитать частные производные именно **аналитически** и **переписать их в код**, а не считать производные численно (через отношение приращения функции к приращению аргумента) - в этих двух случаях могут различаться ответы, поэтому будьте внимательны.

In [ ]:
def grad_f(x): 
    """
    Градиент функциии f, определенной выше.
        param x: np.array(np.float) - массив размерности 2
        return: np.array(np.float)- массив размерности 2
    """

    # 2 * np.sin(x) * np.cos(x)
    return np.sin(2 * x)

In [ ]:
def grad_descent_2d(f, grad_f, lr, num_iter=100, x0=None):
    """
    Функция, которая реализует градиентный спуск в минимум для функции f от двух переменных. 
        param f - скалярная функция двух переменных
        param grad_f - градиент функции f (вектор размерности 2)
        param lr - learning rate алгоритма
        param num_iter - количество итераций градиентного спуска
        return - np.array пар вида (x, f(x))
    """

    if x0 is None:
        x0 = np.random.random(2)
    
    # будем сохранять значения аргументов и значений функции 
    # в процессе градиентного спуска в переменную history
    history = []
    
    # итерация цикла - шаг градиентнго спуска
    curr_x = np.array(x0)
    for iter_num in range(num_iter):
        entry = np.hstack((curr_x, f(curr_x)))
        history.append(entry)
        
        # обновите curr_x
        curr_x -= lr * grad_f(curr_x)

    return np.vstack(history)

Визуализируем точки градиентного спуска на 3D-графике нашей функции. Кружочками будут обозначены точки (тройки $x_1, x_2, f(x_1, x_2)$), по которым ваш алгоритм градиентного спуска двигался к минимуму.

Для того, чтобы нарисовать этот график, мы и сохраняли значения $cur\_x_1, cur\_x_2, f(cur\_x_1, cur\_x_2)$ в `steps` в процессе спуска.

Если у вас правильно написана функция `grad_descent_2d`, то кружочки на картинке должны сходиться к одной из точек минимума функции. Вы можете менять начальные приближения алгоритма, значения `lr` и `num_iter` и получать разные результаты.

In [ ]:
steps = grad_descent_2d(f, grad_f, lr=0.4, num_iter=20)

In [ ]:
# %matplotlib osx

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

path = []

X, Y = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))

fig = plt.figure(figsize=(16, 10))
ax = fig.gca(projection='3d')

ax.plot_surface(X, Y, f([X, Y]), cmap=cm.coolwarm, zorder=2)

# редактируем настройки отображения траектории, сделанной градиентым спуском
ax.plot(xs=steps[:, 0], ys=steps[:, 1], zs=steps[:, 2],
        marker='o', markersize=8, zorder=3, 
        markerfacecolor='red', lw=3, c='green')

ax.set_zlim(0, 5)
ax.view_init(elev=60)
plt.show()

## Линейная регрессия

В данном пункте мы реализуем метод градиентного спуска для задачи регрессии:

$$
Y = XW + B + \varepsilon
$$

Обратите внимание, что $Y$ - матрица размера $n\_samples \times n\_targets$, т.е. для каждого объекта мы будем предсказывать не $1$ число, а $n\_targets$ чисел. Такая модель будет полезна в будущем, когда вы столкнетесь с полносвязным слоем в нейронных сетях. Размерности остальных матриц в формуле согласованы с $Y$!

In [ ]:
X, Y = datasets.make_regression(n_targets=3, n_features=2, noise=10, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

Для отыскания значения параметров мы будем минимизировать **MSE**:

$$
Q(X, Y, W, B) = \frac{1}{n}\|Y - (XW + B)\|_F^2 = \frac{1}{n}\ \mathrm{tr} \left[(Y - XW - B) (Y - XW - B)^{\top}\right]
$$

где $n$ - количество элементов выборки, $\| \cdot \|_F$ — Фробениусова норма матрицы, $\mathrm{tr}$ — след матрицы, $\top$ — оператор транспонирования.

В градиентном спуске на следующем шаге значения параметров получаются из значений на текущем шаге смещением в сторону антиградиента функции потерь:

$$
W_{(k+1)} = W_{(k)} - \eta_k \nabla Q(W_{(k)})
$$

где $\eta_t$ — длина шага (**learning rate**).

Градиент в случае **MSE** выглядит следующим образом:

$$
\begin{aligned}
    \nabla_{W} Q &= \frac{2}{n}X^T(\widehat{Y} - Y)\\
    \nabla_{B} Q &= \frac{2}{n}(\widehat{Y} - Y)
\end{aligned}
$$

где $\widehat{Y} = XW + B$.

У нас есть несколько переменных, чтобы получить **MSE** нужно посчитать сумму квадратов ошибок и поделить на $n$, а не на $nk$, где $n$ - размер выборки, а $k$ - размерность таргета, для **MAE** так же.

In [ ]:
class LinearRegression:
    def __init__(self, l_p_metric=2, seed=42):
        """
        param l_p_metric - задаёт метрику для оптимизации
        Значение 1 соответсвует MAE, 2 — MSE
        param seed - radom_seed для случайной инициализации весов
        """
        
        # используйте np.linalg.norm
        self.metric = lambda y_pred, y: np.mean(np.linalg.norm(y_pred - y, ord=l_p_metric, axis=1) ** l_p_metric)
        self.seed = seed

        self.W = None
        self.b = None

    def init_weights(self, input_size, output_size):
        """
        Инициализирует параметры модели
            param W - матрица размера (input_size, output_size)
            инициализируется рандомными числами из нормального
            распределения со средним 0 и стандартным отклонением 0.01
            param b - вектор размера (1, output_size)
            инициализируется нулями
        """
        
        np.random.seed(self.seed)
        self.W = np.random.normal(loc=0, scale=0.01, size=(input_size, output_size))
        self.b = np.zeros((1, output_size))

    def fit(self, X, y, num_epochs=1000, lr=0.001):
        """
        Обучение модели линейной регрессии методом градиентного спуска
            param X - матрица размера (num_samples, input_size)
            param y - матрица размера (num_samples, output_size)
            param num_epochs - количество итераций градиентного спуска
            param lr - шаг градиентного спуска
            return metrics - вектор значений метрики на каждом шаге градиентного
            спуска. Метрика контролируется параметром l_p_metric в конструкторе
        """
        
        self.init_weights(X.shape[1], y.shape[1])
        metrics = []
        
        for epoch in range(num_epochs):
            y_pred = self.predict(X)
            
            # сделайте вычисления градиентов без циклов,
            # используя только NumPy
            n = X.shape[0]
            
            W_grad = 2 / n * X.T @ (y_pred - y)
            b_grad = 2 * np.mean(y_pred - y, axis=0)        
            
            self.W -= lr * W_grad
            self.b -= lr * b_grad
            
            metrics.append(self.metric(y_pred, y))
            
        return metrics

    def predict(self, X):
        """
        Думаю, тут все понятно. Сделайте свои предсказания :)
        """
        
        return X @ self.W + self.b

In [ ]:
model = LinearRegression()
mse = model.fit(X_train, Y_train)

In [ ]:
# постройте график для MSE
plt.figure(figsize=(15, 8))
plt.plot(mse)
plt.show()

In [ ]:
model = LinearRegression(l_p_metric=1)
mae = model.fit(X_train, Y_train)

In [ ]:
# постройте график для MAE
plt.figure(figsize=(15, 8))
plt.plot(mse)
plt.show()

## Логистическая регрессия

Теперь будем решать задачу классификации при помощи логистической регрессии.

In [ ]:
X, y = datasets.make_blobs(n_samples=10000, n_features=2, centers=2, random_state=42)
y = y[:, np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
plt.figure(figsize=(15, 8))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train[:, 0])
plt.show()

#### Небольшое напоминание по логистической регрессии

Сигмоида:

$$
\sigma(h) = \frac{1}{1 + e^{-h}}
$$

Вероятность принадлежности к классу:

$$
P(y = 1 | x, w) = \sigma(x \cdot w)
$$

Логистическая функция потерь:

$$
L(y, p) = - \frac{1}{m}\sum_{1}^{m}(y_i log(p_i) + (1 - y_i) log(1 - p_i))
$$

Для вывода логистической регрессии удобнее рассматривать бинарную классификацию, где метки классов лежат во множестве $\{0, 1\}$.

Задачу обучения логистической регрессии можно записать следующим образом:

$$
L(y, p)  \to \min_w
$$

Обучение в данном случае сводится к нахождению параметров модели $w$, которое производится с помощью метода градиентного спуска (Gradient Descent, GD). 

Градиентный шаг будет заключаться в обновлении вектора весов по следующей формуле:

$$
w := w - \eta \frac{1}{m} X^T (p - y)
$$

где $\eta > 0$ — размер шага (**learning rate**).

In [ ]:
class LogisticRegressionGD:
    """
    Простая логистическая регрессия с градиентным спуском для задачи бинарной классификации
    """
    
    def __init__(self):
        pass
    
    def __extend_X(self, X):
        """
        Данный метод должен возвращать следующую матрицу:
        X_ext = [1, X], где 1 - единичный вектор.
        Это необходимо для того, чтобы было удобнее производить
        вычисления, т.е. вместо того, чтобы считать X @ W + b
        можно было считать X_ext @ W_ext
        """
        
        return np.hstack((np.ones((X.shape[0], 1)), X))
    
    def init_weights(self, input_size, output_size):
        """
        Инициализирует параметры модели.
        W - матрица размера (input_size, output_size)
        инициализируется рандомными числами из нормального распределения
        со средним 0 и стандартным отклонением 0.01
        """
        
        np.random.seed(42)
        self.W = np.random.normal(loc=0, scale=0.01, size=(input_size, output_size))
        
    def get_loss(self, p, y):
        """
        Данный метод вычисляет логистическую функцию потерь
            param p - вероятности принадлежности к классу 1
            param y - истинные метки
        """
        
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))
                         
    def sigmoid(self, h):
        return 1 / (1 + np.exp(-h))
    
    def get_prob(self, X):
        """
        Данный метод вычисляет P(y = 1 | X, W)
        Возможно, будет удобнее реализовать дополнительный
        метод для вычисления сигмоиды
        """
        
        if X.shape[1] != self.W.shape[0]:
            X = self.__extend_X(X)
        
        return self.sigmoid(X @ self.W)
    
    def get_acc(self, p, y, threshold=0.5):
        """
        Данный метод вычисляет accuracy:
        acc = \frac{1}{len(y)}\sum_{i=1}^{len(y)}{I[y_i == (p_i >= threshold)]}
        """
        
        return np.mean(y == (p >= threshold))

    def fit(self, X, y, num_epochs=100, lr=0.001):
        
        X = self.__extend_X(X)
        self.init_weights(X.shape[1], y.shape[1])
        
        accs = []
        losses = []
          
        for epoch in range(num_epochs):
            p = self.get_prob(X)

            W_grad = X.T @ (p - y) / y.shape[0]
            self.W -= lr * W_grad
            
            # необходимо для стабильности вычислений под логарифмом
            p = np.clip(p, 1e-10, 1 - 1e-10)
            
            log_loss = self.get_loss(p, y)
            losses.append(log_loss)
            acc = self.get_acc(p, y)
            accs.append(acc)
        
        return accs, losses

In [ ]:
model = LogisticRegressionGD()
accs, losses = model.fit(X_train, y_train)

In [ ]:
# постройте графики для accuracy и для loss
plt.figure(figsize=(15, 8))
plt.plot(accs)
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(losses)
plt.show()

## Логистическая регрессия с SGD

Сложность вычисления в случае с **GD** - $O(kn)$, где $k$ - размер вектора признаков, $n$ - размер всей выборки.
В случае, когда выборка очень большая, это решение становится непрактичным.

**GD** заменяют на **SGD** - стохастический градиентный спуск. Он отличается от обычного заменой градиента на несмещённую оценку по одному или нескольким объектам. В этом случае сложность становится  $O(kl)$ , где  $l$  — количество объектов, по которым оценивается градиент,  $l << n$.

In [ ]:
def batch_generator(X, y, batch_size=100):
    """
    Необходимо написать свой генератор батчей.
    Если вы не знаете, что такое генератор, то, возможно,
    вам поможет https://habr.com/ru/post/132554/
    В данном генераторе не надо перемешивать данные
    """
    
    num_samples = X.shape[0]
    # заметьте, что в данном случае, если num_samples не делится на
    # batch_size, то последние элементы никогда не попадут в обучение
    # в данном случае нас это не волнует
    num_batches = num_samples // batch_size
    for i in range(num_batches):
        # необходимо отдать batch_size обьектов и соответствующие им target
        ind = slice(i * batch_size, (i + 1) * batch_size)
        yield X[ind], y[ind]


class LogisticRegressionSGD:
    def __init__(self):
        pass
    
    def __extend_X(self, X):
        """
        Данный метод должен возвращать следующую матрицу:
        X_ext = [1, X], где 1 - единичный вектор.
        Это необходимо для того, чтобы было удобнее производить
        вычисления, т.е. вместо того, чтобы считать X @ W + b
        можно было считать X_ext @ W_ext
        """
        
        return np.hstack((np.ones((X.shape[0], 1)), X))

    def init_weights(self, input_size, output_size):
        """
        Инициализирует параметры модели.
        W - матрица размера (input_size, output_size)
        инициализируется рандомными числами из нормального распределения
        со средним 0 и стандартным отклонением 0.01
        """
        
        np.random.seed(42)
        self.W = np.random.normal(loc=0, scale=0.01, size=(input_size, output_size))
        
    def get_loss(self, p, y):
        """
        Данный метод вычисляет логистическую функцию потерь
            param p - вероятности принадлежности к классу 1
            param y - истинные метки
        """
        
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))
    
    def sigmoid(self, h):
        return 1 / (1 + np.exp(-h))
    
    def get_prob(self, X):
        """
        Данный метод вычисляет P(y = 1 | X, W)
        Возможно, будет удобнее реализовать дополнительный
        метод для вычисления сигмоиды
        """
        
        if X.shape[1] != self.W.shape[0]:
            X = self.__extend_X(X)
        
        return self.sigmoid(X @ self.W)
    
    def get_acc(self, p, y, threshold=0.5):
        """
        Данный метод вычисляет accuracy:
        acc = \frac{1}{len(y)}\sum_{i=1}^{len(y)}{I[y_i == (p_i >= threshold)]}
        """
        
        return np.mean(y == (p >= threshold))

    def fit(self, X, y, num_epochs=10, lr=0.001):
        
        X = self.__extend_X(X)
        self.init_weights(X.shape[1], y.shape[1])
        
        accs = []
        losses = []
        for epoch in range(num_epochs):
            gen = batch_generator(X, y)
            for X_, y_ in gen:
                p = self.get_prob(X_)

                W_grad = X_.T @ (p - y_) / y_.shape[0]
                self.W -= lr * W_grad

                # необходимо для стабильности вычислений под логарифмом
                p = np.clip(p, 1e-10, 1 - 1e-10)

                log_loss = self.get_loss(p, y_)
                losses.append(log_loss)
                acc = self.get_acc(p, y_)
                accs.append(acc)
        
        return accs, losses

In [ ]:
model = LogisticRegressionSGD()
accs, losses = model.fit(X_train, y_train)

In [ ]:
# постройте графики для accuracy и для loss
plt.figure(figsize=(15, 8))
plt.plot(accs)
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(losses)
plt.show()

В данном случае модель тренируется значительно дольше, чем c **Gradient Decent**. Попробуйте объяснить, почему так происходит.